# Mango to do NAS using MNIST NN

In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

- Some configs

In [3]:
import tensorflow as tf 
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

# Prepare the data

In [4]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [5]:
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [6]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
def objective_NN(l1_f=8, l1_k=3, l2_f=8, l2_k=3, Drop=0.5, batch_size=128, epochs=15):
    
    model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(l1_f, kernel_size=(l1_k, l1_k), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(l2_f, kernel_size=(l2_k, l2_k), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(Drop),
        layers.Dense(num_classes, activation="softmax")
    ])
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=0)
    
    score = model.evaluate(x_test, y_test, verbose=0)
    
    return score[1]

In [8]:
from mango.tuner import Tuner
from scipy.stats import uniform

# NAS configurations to search from

In [9]:
# Number of filters, size of kernel, dropouts and batch size


param_dict = {
    'l1_f': range(4,33),
    'l1_k': range(2,6),
    'l2_f': range(4,33),
    'l2_k': range(2,6),
    'Drop': uniform(0, 0.5),
    'batch_size': [64, 128]
}

# Save the results

In [10]:
import pickle 

# Save pickle results
def save_res(data, file_name):
    pickle.dump( data, open( file_name, "wb" ) )

# Objective functions

In [11]:
import time

def objfunc(args_list):

    objective_evaluated = []
    
    start_time = time.time()
    
    for hyper_par in args_list:
        Drop = hyper_par['Drop']
        l1_f = hyper_par['l1_f']
        l1_k = hyper_par['l1_k']
        l2_f = hyper_par['l1_f']
        l2_k = hyper_par['l1_k']
        
        batch_size = hyper_par['batch_size']
        
        #print(hyper_par)
        objective = objective_NN(l1_f=l1_f,l1_k=l1_k, l2_f=l2_f,l2_k=l2_k, Drop=Drop, batch_size=batch_size)
        objective_evaluated.append(objective)
        
        end_time = time.time()
        print('objective:', objective, ' time:',end_time-start_time)
        
    return objective_evaluated

# Mango search configurations

In [12]:
conf_Dict = dict()
conf_Dict['batch_size'] = 1
conf_Dict['num_iteration'] = 50
conf_Dict['domain_size'] =20000
conf_Dict['initial_random']= 5

In [14]:
tuner = Tuner(param_dict, objfunc,conf_Dict)

In [15]:
num_of_tries = 30
all_runs = []

# Run the Mango

In [17]:
for i in range(num_of_tries):
    results = tuner.maximize()
    
    #print('best parameters:',results['best_params'])
    #print('best objective:',results['best_objective'])

    all_runs.append(results)
    
    # saving the results
    save_res(all_runs,'mnist_mango.p')
    